In [ ]:
# Simple example that uses prepared draw function
import webgpu.jupyter
from ngsolve import *

m = Mesh(unit_square.GenerateMesh(maxh=0.5))

webgpu.jupyter.Draw(x, m)

In [ ]:
# more verbose example with custom render function implementation

import webgpu.jupyter
from ngsolve import *

def render(data):
    """This is the setup function running in pyodide (with access to the html dom and JS)"""
    print("hello from pyodide")
    import pyodide.ffi
    import ngsolve as ngs
    from webgpu.jupyter import gpu
    from webgpu.mesh import create_mesh_buffers, create_function_value_buffers, MeshRenderObject

    # unpack the data argument
    mesh = data['mesh']
    cf = data['cf']
    order = data['order']

    # Copy mesh/function data to the GPU
    region = mesh.Region(ngs.VOL)
    n_trigs, buffers = create_mesh_buffers(
        gpu.device, region
    )
    buffers = buffers | create_function_value_buffers(
        gpu.device, cf, region, order
    )
    
    mesh_object = MeshRenderObject(gpu, buffers, n_trigs)

    frame_counter = 0
    import time
    t0 = time.time()
    def render_function(t):
        """This function is called for each frame"""
        nonlocal frame_counter

        # change colormap for some animation
        gpu.uniforms.colormap.min = 0.3 + 0.3*ngs.sin(4*(time.time()-t0))
        gpu.uniforms.colormap.max = 1.0 - gpu.uniforms.colormap.min
        gpu.uniforms.update_buffer()

        # render mesh_object
        encoder = gpu.device.createCommandEncoder()
        mesh_object.render(encoder)
        gpu.device.queue.submit([encoder.finish()])

        # request next frame (for animation, not necessary otherwise)
        # stop after 200 frames to avoid endless loop
        frame_counter += 1
        if frame_counter < 200:
            js.requestAnimationFrame(render_function)

    # we need to create a proxy object to pass a python function as JS callback
    render_function = pyodide.ffi.create_proxy(render_function)

    # set callback to trigger a new frame on click/mouse move etc.
    gpu.input_handler.render_function = render_function

    # request first frame
    js.requestAnimationFrame(render_function)

m = Mesh(unit_square.GenerateMesh(maxh=.2))
data = {
    "mesh": m,
    "cf": sin(10*x)*sin(10*y),
    "order": 6
}

# "data" will be passed as argument to render on the pyodide side
webgpu.jupyter.DrawCustom(data, render)